In [2]:
!pip uninstall -y pyarrow datasets
!pip install --no-use-pep517 pyarrow
!pip install datasets
!pip install torch transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install scikit-learn

Found existing installation: pyarrow 15.0.2
Uninstalling pyarrow-15.0.2:
  Successfully uninstalled pyarrow-15.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 55.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 92.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 58.5

In [3]:
import os

import pandas as pd
import s3fs
import zipfile

## Récupérer les données d'un challenge

In [4]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [5]:
# Lister les fichiers d'un challenge
fs.ls("civel/diffusion/hackathon-minarm-2024/AIVSAI/hack_train.csv")

['civel/diffusion/hackathon-minarm-2024/AIVSAI/hack_train.csv']

In [6]:
# Télécharger les données dans le service
PATH_IN = 'civel/diffusion/hackathon-minarm-2024/AIVSAI/hack_train.csv'
fs.download(PATH_IN, 'data/hack_train.csv')

[None]

In [7]:
df = pd.read_csv('data/hack_train.csv')

In [8]:
df

,text,label,src
0,Little disclaimer: this deals with US laws and...,1,cmv_human
1,"Read: Mentally Retarded Downs. See, we've got ...",1,cmv_human
2,"If any of you frequent rbadhistory, there is a...",1,cmv_human
3,"I believe in a flat tax system, where everyone...",1,cmv_human
4,"Edit: Ok guy's, my views have been changed on ...",1,cmv_human
...,...,...,...
56814,We consider the recovery of a source term f (x...,1,sci_gen_human
56815,"Self-supervised learning (SlfSL), aiming at le...",1,sci_gen_human
56816,Recurrent neural networks (RNNs) have achieved...,1,sci_gen_human
56817,Deep reinforcement learning (DRL) is a booming...,1,sci_gen_human


## Model

In [9]:
from transformers import RobertaTokenizer
from datasets import Dataset, DatasetDict

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

# Create a Hugging Face Dataset from the data (if not already in one)
data_dict = {
    "texts": df['text'],
    "labels": df['label']
}
hf_dataset = Dataset.from_dict(data_dict)

# Define the preprocessing function to tokenize the data
def preprocess_function(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(examples['texts'], padding="max_length", truncation=True, max_length=512)

    # Prepare the dictionary correctly.
    return {
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': examples['labels']
    }

# Apply the preprocessing function to the dataset
tokenized_datasets = hf_dataset.map(preprocess_function, batched=True)

# Display the first few processed entries to verify
print(tokenized_datasets.select(range(2)))  # Select the first two entries for display

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Map:   0%|          | 0/56819 [00:00<?, ? examples/s]

Dataset({
    features: ['texts', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 2
})


In [10]:
!nvidia-smi

Wed Apr 24 15:40:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 PCIe               Off | 00000000:B5:00.0 Off |                    0 |
| N/A   39C    P0              53W / 350W |      4MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [11]:
from transformers import DistilBertTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.cuda

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

# Split the dataset
train_test_split = tokenized_datasets.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_val_dataset = train_test_split['test']

test_val_split = test_val_dataset.train_test_split(test_size=0.5, seed=42)
eval_dataset = test_val_split['train']
test_dataset = test_val_split['test']

# Load the model
model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=2)
print(model.device)

# Check for GPU availability and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model.device)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100
)

# Define the metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='binary')
    return {
        'accuracy': acc,
        'f1_score': f1,
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Save the trained model
torch.save(model.state_dict(), 'model_dataset2.pth')

True
1
0


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cpu
cuda:0


Step,Training Loss,Validation Loss,Accuracy,F1 Score
100,0.688000,0.691995,0.501760,0.668229
200,0.662900,0.720877,0.542415,0.192045
300,0.687400,0.515506,0.785639,0.790361
400,0.369500,0.508878,0.766807,0.744159
500,0.270500,0.886427,0.656635,0.494690
600,0.591600,0.402167,0.822246,0.831047
700,0.356600,0.460161,0.789511,0.760608
800,0.354600,0.615505,0.821718,0.804704
900,0.427500,0.373981,0.839317,0.839853
1000,0.302100,0.499210,0.812742,0.781699


In [13]:
# Set the model to evaluation mode
model.eval()

# Move the model to the appropriate device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Evaluate the model on the test dataset
predictions = trainer.predict(test_dataset)
test_metrics = compute_metrics(predictions)

# Printing all the metrics
for metric_name, metric_value in test_metrics.items():
    print(f"Test {metric_name.capitalize()}:", metric_value)

Test Accuracy: 0.8796198521647307
Test F1_score: 0.8715734134434848


In [25]:
import torch
from torch.nn.functional import softmax

# Check for GPU availability and set the device accordingly
device = torch.device("cpu" if torch.cuda.is_available() else "cuda")
model.to(device)  # Move the model to the appropriate device

# Function to make a prediction on a single sentence
def predict(sentence):
    # Tokenize the sentence so it matches the format expected by the model
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Predict
    with torch.no_grad():  # Disable gradient calculation to speed up the process and reduce memory usage
        logits = model(**inputs).logits

    # Apply softmax to logits to get probabilities
    probabilities = softmax(logits, dim=1)

    # Assuming we have two classes, 0 and 1, and class 0 is the 'negative' class
    prediction = probabilities.argmax().item()  # Get the index of the highest probability
    return {"class": prediction, "probabilities": probabilities.tolist()[0]}

# Example usage
user_sentence = "Yes, it is possible to be subject to a cash withdrawal even if you do not use an ATM. There are several ways that this could happen:Debit card transactions: If you make a purchase using your debit card, the merchant may automatically withdraw the amount of the purchase from your checking account. This is essentially the same as making a cash withdrawal.Bank fees: Some banks charge fees for maintaining an account or for using certain services. These fees may be automatically withdrawn from your account on a regular basis.Automatic payments: If you have set up automatic payments for bills or other expenses, the amount of the payment will be withdrawn from your account when it is due.Check payments: If you write a check to pay for something, the recipient may deposit the check and withdraw the funds from your account.Electronic transfers: You may also be subject to a cash withdrawal if you authorize an electronic transfer of funds from your account to another account.In summary, there are many ways that you could be subject to a cash withdrawal even if you do not use an ATM. It is important to carefully track your account balance and be aware of any automatic transactions or payments that may be taking place."
result = predict(user_sentence)
print("Predicted Class:", result["class"])
print("Probabilities:", result["probabilities"])

Predicted Class: 0
Probabilities: [0.9990405440330505, 0.000959432334639132]
